In [4]:
from img_data_class import ImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import torch
from torch import nn
import matplotlib.pyplot as plt
from model_class import InpaintingVAE
import torch.optim as optim
import torch.nn.functional as F
import tqdm

c:\Users\Tomislav\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
EPOCHS = 1
BATCH_SIZE = 32


In [6]:
tr = transforms.Compose(
    [
     transforms.Resize((256,256)),
     transforms.ToTensor()
    ],
    )
random_erase_tr = transforms.RandomErasing(1)

In [7]:
train = ImageDataset('data/train',tr)

In [8]:
train_dl = DataLoader(train,BATCH_SIZE,shuffle=True)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
model = InpaintingVae(3,2,device).to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [28]:
checkpoint = torch.load('models/vae_2022_12_25_16_56_17.pt')

In [9]:
def loss_function(recon_x, x, mu, log_var):
    bce = nn.CrossEntropyLoss(reduction='sum')
    BCE = bce(recon_x.flatten(1), x.flatten(1))
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD
def augment_image(random_erase_fn,img):
    img_list = [random_erase_fn(i) for i in img]
    return torch.stack(img_list)

In [10]:
for epoch in range(EPOCHS):
    train_epoch_loss = 0
    for batch_idx,img in tqdm.tqdm(enumerate(train_dl),total=len(train_dl)):
        model.train()
        #plt.imshow(make_grid(img).permute(1,2,0))
        input_img = augment_image(random_erase_tr,img).to(device)
        img = img.to(device)
        
        optimizer.zero_grad()
        recon_img, mu , log_var = model(input_img)
        loss = loss_function(recon_img, img, mu, log_var)
        train_epoch_loss += loss.item()
        loss.backward()
        optimizer.zero_grad()
        

 28%|██▊       | 1402/5065 [24:31<1:04:04,  1.05s/it]


KeyboardInterrupt: 

In [ ]:
import os

In [ ]:
len(os.listdir('data/train/'))

162079

In [ ]:
len(os.listdir('data/test/'))

40520

In [1]:
import time

In [3]:
time.strftime("%Y_%m_%d_%H_%M_%S")

'2022_12_25_16_31_46'